# ML Training - Prédiction de gravité des ACCIDENTS

## Contexte du projet

Ce notebook fait partie d'un projet de prédiction de la gravité des accidents routiers en France. L'objectif est de construire un modèle ML capable de prédire si un accident sera **grave** ou **non grave** à partir de caractéristiques contextuelles (lieu, moment, conditions, véhicules impliqués).

## Objectif spécifique

**Prédire la gravité d'un ACCIDENT** (et non d'un usager individuel).  
La gravité d'un accident est définie comme la **gravité maximale** parmi toutes les personnes impliquées :
- Si au moins une personne est tuée ou hospitalisée → accident **grave**
- Sinon → accident **non grave**

## Critères de sélection du modèle

| Critère | Métrique | Justification |
|---------|----------|---------------|
| **Principal** | F1-Score | Équilibre entre précision et rappel, adapté aux classes déséquilibrées |
| **Secondaire** | AUC | Départage en cas d'égalité sur F1, mesure la capacité discriminante |

**Pourquoi le F1 ?**  
Le F1-Score est la moyenne harmonique de la précision et du rappel. Il pénalise fortement les modèles qui sacrifient l'un pour l'autre, ce qui est crucial pour notre cas d'usage :
- **Faux négatifs** (accident grave non détecté) : risque de sous-estimer les ressources de secours
- **Faux positifs** (fausse alerte) : mobilisation inutile des secours

## Workflow du notebook

```
1. Chargement des données
   └── Dataset ACCIDENT (164k accidents)

2. Entraînement initial (baseline)
   ├── Target binaire (grave/non-grave)
   └── Target ordonné (4 niveaux de gravité)

3. Comparaison Binary vs Ordered
   └── Justification du choix binaire

4. Feature Importance
   └── Identification des variables clés

5. Fine-tuning
   ├── GridSearchCV (hyperparamètres optimaux)
   ├── Comparaison algorithmes (RF, XGBoost, LightGBM, CatBoost)
   └── Sélection du meilleur modèle

6. Sauvegarde des modèles
   └── Modèles prêts pour l'API
```

## Targets disponibles

| Target | Type | Classes | Description |
|--------|------|---------|-------------|
| `grav_binary` | Binaire | 0, 1 | 0 = non grave, 1 = grave (hospitalisé ou tué) |
| `grav_ordered` | Multiclasse | 0, 1, 2, 3 | Indemne, Blessé léger, Hospitalisé, Tué |

---

## 1. Configuration et imports

### Bibliothèques utilisées

| Catégorie | Bibliothèques | Usage |
|-----------|---------------|-------|
| **Data** | pandas, numpy | Manipulation des données |
| **Visualisation** | plotly | Graphiques interactifs (matrices de confusion, ROC, barres) |
| **ML - Preprocessing** | sklearn (StandardScaler, LabelEncoder, SimpleImputer) | Normalisation, encodage, imputation |
| **ML - Modèles** | RandomForest, XGBoost, LightGBM, CatBoost | Algorithmes de classification |
| **ML - Évaluation** | sklearn.metrics | F1, AUC, confusion matrix, etc. |
| **Persistance** | joblib | Sauvegarde des modèles entraînés |

In [1]:
# === IMPORTS ===
import os
import pandas as pd
import numpy as np
import joblib

from ml_config import nb_workers, base_estimators

# Visualisation
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Machine Learning
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    auc,
)

# === CONFIGURATION ===
# Seed pour reproductibilité : garantit les mêmes résultats à chaque exécution
RANDOM_STATE = 42

print("Imports OK")

Imports OK


---

## 2. Chargement des données

### Source des données

Les données proviennent du fichier `dataset_accident.parquet` généré par le notebook de préparation (`2-data-segmentation.ipynb`).

**Caractéristiques du dataset ACCIDENT :**
- **Grain** : 1 ligne = 1 accident (identifié par `Num_Acc`)
- **Gravité** : Agrégée au niveau de l'accident (= gravité max des personnes impliquées)
- **Features** : Contexte temporel, géographique, météo, véhicules impliqués

In [2]:
# Fonction de chargement (supporte Parquet et CSV)
def load_dataset(name, base_path="data/output"):
    """
    Charge un dataset depuis Parquet.
    Parquet est préféré car plus rapide et compact.
    """
    parquet_path = f"{base_path}/{name}.parquet"

    if os.path.exists(parquet_path):
        df = pd.read_parquet(parquet_path)
        print(f"{name}: chargé depuis Parquet ({df.shape[0]:,} lignes, {df.shape[1]} colonnes)")
    else:
        raise FileNotFoundError(f"Dataset {name} non trouvé")
    return df


# Chargement du dataset ACCIDENT
df_accident = load_dataset("dataset_accident")

# Aperçu des données
print(f"\n📊 Colonnes disponibles ({len(df_accident.columns)}):")
print(df_accident.columns.tolist())

dataset_accident: chargé depuis Parquet (263,356 lignes, 11 colonnes)

📊 Colonnes disponibles (11):
['grav_ordered', 'grav_binary', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'agg', 'vma', 'impl_vehicule_leger', 'impl_poids_lourd', 'impl_pieton']


---

## 3. Fonctions utilitaires

### Pipeline de prétraitement

Le pipeline ML suit ces étapes :

```
Données brutes → Imputation → Scaling → Modèle → Prédictions
```

1. **Imputation** (SimpleImputer) : Remplace les valeurs manquantes par la médiane
   - *Pourquoi la médiane ?* Robuste aux outliers, contrairement à la moyenne
   
2. **Scaling** (StandardScaler) : Normalise les features (moyenne=0, écart-type=1)
   - *Pourquoi ?* Améliore la convergence des algorithmes à base de gradient (XGBoost, etc.)

### Gestion des classes déséquilibrées

Les accidents **graves** sont minoritaires (~35%). Pour éviter que le modèle favorise la classe majoritaire :
- RandomForest : `class_weight='balanced'`
- XGBoost : `scale_pos_weight` (ratio négatifs/positifs)

In [3]:
# === FONCTIONS DE PRÉPARATION DES DONNÉES ===


def prepare_data(df, target_col, exclude_cols=None):
    """
    Prépare les features (X) et la target (y) pour l'entraînement.

    - Exclut automatiquement toutes les colonnes de gravité (pour éviter le data leakage)
    - Encode les variables catégorielles en entiers

    Returns: X (DataFrame), y (Series)
    """
    if exclude_cols is None:
        exclude_cols = []

    # Colonnes de gravité à exclure (évite le data leakage)
    target_cols = ["grav_ordered", "grav_binary"]
    cols_to_drop = target_cols + exclude_cols
    cols_to_drop = [c for c in cols_to_drop if c in df.columns]

    X = df.drop(columns=cols_to_drop)
    y = df[target_col]

    # Encodage des variables catégorielles
    for col in X.select_dtypes(include=["object"]).columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))

    return X, y


# === FONCTIONS DE MODÉLISATION ===
class VerboseImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy="median"):
        self.strategy = strategy
        self.imputer = SimpleImputer(strategy=strategy)
        self.feature_names_ = None

    def fit(self, X, y=None):
        # Stocker les noms de colonnes si disponibles
        if hasattr(X, "columns"):
            self.feature_names_ = X.columns.tolist()
        self.imputer.fit(X)
        return self

    def transform(self, X):
        if hasattr(X, "isna"):
            missing_per_col = X.isna().sum()
        else:
            missing_per_col = np.isnan(X).sum(axis=0)

        total = missing_per_col.sum() if hasattr(missing_per_col, "sum") else sum(missing_per_col)

        if total > 0:
            print(f"Valeurs imputées ({self.strategy}) : {total}")
            # Détail par colonne
            if hasattr(X, "columns"):
                for col in X.columns:
                    if X[col].isna().sum() > 0:
                        print(f"  - {col}: {X[col].isna().sum()}")
            elif self.feature_names_:
                for i, name in enumerate(self.feature_names_):
                    count = missing_per_col[i] if hasattr(missing_per_col, "__getitem__") else missing_per_col
                    if count > 0:
                        print(f"  - {name}: {count}")

        return self.imputer.transform(X)


def create_pipeline(model):
    """
    Crée un pipeline sklearn : Imputation → Scaling → Modèle.
    Permet d'encapsuler tout le preprocessing avec le modèle.
    """
    return Pipeline([("imputer", VerboseImputer(strategy="median")), ("scaler", StandardScaler()), ("model", model)])


def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Entraîne un modèle et calcule les métriques d'évaluation.

    Returns:
        - results: dict avec accuracy, precision, recall, f1, auc
        - y_pred: prédictions sur le test set
        - y_proba: probabilités (pour ROC curve)
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Probabilités pour la courbe ROC
    y_proba = None
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)

    # Calcul des métriques
    results = {
        "model": model_name,
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average="weighted", zero_division=0),
        "recall": recall_score(y_test, y_pred, average="weighted", zero_division=0),
        "f1": f1_score(y_test, y_pred, average="weighted", zero_division=0),
    }

    # AUC uniquement pour la classification binaire
    if len(np.unique(y_test)) == 2 and y_proba is not None:
        results["auc"] = roc_auc_score(y_test, y_proba[:, 1])

    return results, y_pred, y_proba


def evaluate_catboost(catboost_model, X_train, X_test, y_train, y_test, model_name):
    """
    Évalue CatBoost SANS Pipeline sklearn.

    Note: CatBoost est incompatible avec sklearn Pipeline depuis sklearn 1.6+
    (problème de sérialisation). On applique donc le preprocessing manuellement.
    """
    # Preprocessing manuel
    imputer = SimpleImputer(strategy="median")
    X_train_imp = imputer.fit_transform(X_train)
    X_test_imp = imputer.transform(X_test)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imp)
    X_test_scaled = scaler.transform(X_test_imp)

    # Entraînement et prédiction
    catboost_model.fit(X_train_scaled, y_train)
    y_pred = catboost_model.predict(X_test_scaled)

    y_proba = None
    if hasattr(catboost_model, "predict_proba"):
        y_proba = catboost_model.predict_proba(X_test_scaled)

    # Métriques
    results = {
        "model": model_name,
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average="weighted", zero_division=0),
        "recall": recall_score(y_test, y_pred, average="weighted", zero_division=0),
        "f1": f1_score(y_test, y_pred, average="weighted", zero_division=0),
    }

    if len(np.unique(y_test)) == 2 and y_proba is not None:
        results["auc"] = roc_auc_score(y_test, y_proba[:, 1])

    return results, y_pred, y_proba


# === FONCTIONS DE VISUALISATION ===


def plot_confusion_matrix(y_true, y_pred, model_name, labels=None):
    """Affiche une matrice de confusion interactive avec Plotly."""
    cm = confusion_matrix(y_true, y_pred)

    if labels is None:
        labels = [str(i) for i in sorted(np.unique(y_true))]

    # Pourcentages par ligne (= par vraie classe)
    cm_percent = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] * 100

    # Annotations : valeur absolue + pourcentage
    text_annotations = []
    for i in range(len(cm)):
        row = []
        for j in range(len(cm[0])):
            row.append(f"{cm[i][j]}<br>({cm_percent[i][j]:.1f}%)")
        text_annotations.append(row)

    fig = go.Figure(
        data=go.Heatmap(
            z=cm, x=labels, y=labels, text=text_annotations, texttemplate="%{text}", colorscale="Blues", showscale=True
        )
    )

    fig.update_layout(
        title=f"Matrice de Confusion - {model_name}",
        xaxis_title="Prédiction",
        yaxis_title="Réalité",
        width=500,
        height=450,
    )
    fig.show()
    return cm


def plot_roc_curve(y_true, y_proba, model_name):
    """Affiche la courbe ROC avec l'AUC (classification binaire uniquement)."""
    if len(np.unique(y_true)) != 2:
        print(f"ROC non disponible pour {model_name}: pas binaire")
        return None

    if y_proba is None:
        print(f"ROC non disponible pour {model_name}: pas de probabilités")
        return None

    y_score = y_proba[:, 1] if len(y_proba.shape) > 1 else y_proba
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=fpr, y=tpr, mode="lines", name=f"{model_name} (AUC = {roc_auc:.3f})", line=dict(color="blue", width=2)
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[0, 1], y=[0, 1], mode="lines", name="Random (AUC = 0.5)", line=dict(color="gray", width=1, dash="dash")
        )
    )

    fig.update_layout(
        title=f"Courbe ROC - {model_name} (AUC = {roc_auc:.3f})",
        xaxis_title="Taux de Faux Positifs (FPR)",
        yaxis_title="Taux de Vrais Positifs (TPR)",
        width=550,
        height=450,
        showlegend=True,
    )
    fig.show()
    return roc_auc


print("Fonctions définies")

Fonctions définies


---

## 4. Entraînement initial (Baseline)

### Objectif

Établir une **baseline** de performance avec des modèles standards (RandomForest, XGBoost) sur les deux types de targets :
- **Binary** : 2 classes (grave / non grave)
- **Ordered** : 4 classes (indemne, léger, hospitalisé, tué)

### Stratégie d'évaluation

| Paramètre | Valeur | Justification |
|-----------|--------|---------------|
| Split train/test | 80/20 | Standard pour ~160k échantillons |
| Stratification | Oui | Préserve la distribution des classes |
| n_estimators | 100 | Baseline raisonnable (optimisé plus tard) |

### Algorithmes testés

1. **RandomForest** : Ensemble de decision trees, robuste et interprétable
2. **XGBoost** : Gradient boosting, souvent plus performant mais moins interprétable

### 4.1 Target binaire (`grav_binary`)

**Définition :** 
- `0` = Accident non grave (indemne ou blessé léger)
- `1` = Accident grave (hospitalisé ou tué)

**Pourquoi privilégier le target binaire ?**
- Plus simple à prédire (2 classes vs 4)
- Décision claire pour l'API (déclencher une alerte ou non)
- Meilleur équilibre des classes (~35% grave vs ~65% non grave)

In [4]:
# === PRÉPARATION DES DONNÉES ===
X_acc, y_acc_bin = prepare_data(df_accident, "grav_binary")

# Split stratifié : préserve le ratio grave/non-grave dans train et test
X_train_acc, X_test_acc, y_train_acc, y_test_acc = train_test_split(
    X_acc,
    y_acc_bin,
    test_size=0.2,  # 20% pour le test
    random_state=RANDOM_STATE,  # Reproductibilité
    stratify=y_acc_bin,  # Même distribution de classes
)

print(f"Features utilisées ({len(X_acc.columns)}):")
print(f"{X_acc.columns.tolist()}")
print(f"\nTaille des ensembles:")
print(f"Train: {len(X_train_acc):,} | Test: {len(X_test_acc):,}")
print(f"\nDistribution de la target:")
print(f"Non grave (0): {(y_acc_bin == 0).sum():,} ({(y_acc_bin == 0).mean() * 100:.1f}%)")
print(f"Grave (1): {(y_acc_bin == 1).sum():,} ({(y_acc_bin == 1).mean() * 100:.1f}%)")

Features utilisées (9):
['est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'agg', 'vma', 'impl_vehicule_leger', 'impl_poids_lourd', 'impl_pieton']

Taille des ensembles:
Train: 210,684 | Test: 52,672

Distribution de la target:
Non grave (0): 170,849 (64.9%)
Grave (1): 92,507 (35.1%)


In [5]:
# === ENTRAÎNEMENT DES MODÈLES BASELINE ===
print(f"weight_scaling: {len(y_train_acc[y_train_acc == 0])} / {len(y_train_acc[y_train_acc == 1])}")

# Définition des modèles avec gestion du déséquilibre de classes
models_acc = {
    "RandomForest": create_pipeline(
        RandomForestClassifier(
            n_estimators=base_estimators,
            random_state=RANDOM_STATE,
            n_jobs=nb_workers,  # Utilise tous les CPU
            class_weight="balanced",  # Pondère les classes selon leur fréquence
        )
    ),
    "XGBoost": create_pipeline(
        XGBClassifier(
            n_estimators=base_estimators,
            random_state=RANDOM_STATE,
            n_jobs=nb_workers,
            # scale_pos_weight = ratio négatifs/positifs pour équilibrer
            scale_pos_weight=len(y_train_acc[y_train_acc == 0]) / len(y_train_acc[y_train_acc == 1]),
        )
    ),
}

# Entraînement et évaluation
results_acc_bin = []
predictions_acc_bin = {}

print("🚀 Entraînement des modèles baseline...\n")
for name, model in models_acc.items():
    res, y_pred, y_proba = evaluate_model(model, X_train_acc, X_test_acc, y_train_acc, y_test_acc, name)
    results_acc_bin.append(res)
    predictions_acc_bin[name] = {"y_pred": y_pred, "y_proba": y_proba}
    print(f"   {name:15} → F1={res['f1']:.3f} | AUC={res.get('auc', 0):.3f}")

# Tableau récapitulatif
print("\n📊 Résultats baseline (target binaire):")
pd.DataFrame(results_acc_bin).round(3)

# Sauvegarder les resultats pour le notebook de fine-tuning
joblib.dump(results_acc_bin, "models/results_acc_bin_baseline.pkl")
print("Resultats sauvegardes dans models/results_acc_bin_baseline.pkl")

weight_scaling: 136679 / 74005
🚀 Entraînement des modèles baseline...

   RandomForest    → F1=0.683 | AUC=0.705
   XGBoost         → F1=0.686 | AUC=0.709

📊 Résultats baseline (target binaire):
Resultats sauvegardes dans models/results_acc_bin_baseline.pkl


### 4.1.1 Visualisations des résultats baseline

Pour chaque modèle, on affiche :
- **Matrice de confusion** : répartition des prédictions (TP, TN, FP, FN)
- **Courbe ROC** : compromis entre sensibilité (TPR) et spécificité (1-FPR)

**Lecture de la matrice de confusion :**
| Zone | Signification |
|------|---------------|
| Diagonale | Prédictions correctes |
| Haut-droite (FP) | Fausses alertes |
| Bas-gauche (FN) | Cas graves manqués |

In [6]:
# === VISUALISATIONS : Matrices de confusion et courbes ROC ===
labels_binary = ["Non grave (0)", "Grave (1)"]

for name, preds in predictions_acc_bin.items():
    print(f"\n{'=' * 60}")
    print(f"📊 {name} - Visualisations")
    print("=" * 60)

    # Matrice de confusion
    plot_confusion_matrix(y_test_acc, preds["y_pred"], f"{name}", labels_binary)

    # Courbe ROC
    plot_roc_curve(y_test_acc, preds["y_proba"], f"{name}")


📊 RandomForest - Visualisations



📊 XGBoost - Visualisations


### 4.2 Target ordonné (`grav_ordered`) - Multiclasse

**Définition :** 4 niveaux de gravité
- `0` = Indemne
- `1` = Blessé léger
- `2` = Hospitalisé  
- `3` = Tué

**Problème attendu :** Classes très déséquilibrées (Indemne < 1%, Tué ~6%)

*Ce modèle est entraîné pour comparaison, mais le target binaire sera privilégié.*

In [7]:
# === PRÉPARATION DES DONNÉES (target ordered) ===
X_acc_ord, y_acc_ord = prepare_data(df_accident, "grav_ordered")
X_tr_acc_ord, X_te_acc_ord, y_tr_acc_ord, y_te_acc_ord = train_test_split(
    X_acc_ord, y_acc_ord, test_size=0.2, random_state=RANDOM_STATE, stratify=y_acc_ord
)

# Afficher la distribution (déséquilibre visible)
print("⚖️  Distribution du target ordered:")
for val, count in y_acc_ord.value_counts().sort_index().items():
    labels = ["Indemne", "Blessé léger", "Hospitalisé", "Tué"]
    pct = count / len(y_acc_ord) * 100
    bar = "█" * int(pct / 2)
    print(f"   {val} ({labels[val]:14}) : {count:6,} ({pct:5.1f}%) {bar}")

⚖️  Distribution du target ordered:
   0 (Indemne       ) :    627 (  0.2%) 
   1 (Blessé léger  ) : 170,222 ( 64.6%) ████████████████████████████████
   2 (Hospitalisé   ) : 78,023 ( 29.6%) ██████████████
   3 (Tué           ) : 14,484 (  5.5%) ██


In [8]:
# === ENTRAÎNEMENT DES MODÈLES (target ordered) ===
models_acc_ord = {
    "RandomForest": create_pipeline(
        RandomForestClassifier(
            n_estimators=base_estimators, random_state=RANDOM_STATE, n_jobs=nb_workers, class_weight="balanced"
        )
    ),
    "XGBoost": create_pipeline(
        XGBClassifier(
            n_estimators=base_estimators,
            random_state=RANDOM_STATE,
            n_jobs=nb_workers,
            objective="multi:softmax",  # Multiclasse
            num_class=4,
        )
    ),
}

results_acc_ord = []
predictions_acc_ord = {}

print("🚀 Entraînement des modèles (target ordered)...\n")
for name, model in models_acc_ord.items():
    res, y_pred, y_proba = evaluate_model(model, X_tr_acc_ord, X_te_acc_ord, y_tr_acc_ord, y_te_acc_ord, name)
    results_acc_ord.append(res)
    predictions_acc_ord[name] = {"y_pred": y_pred, "y_proba": y_proba}
    print(f"   {name:15} → F1={res['f1']:.3f} | Accuracy={res['accuracy']:.3f}")

# Note: AUC non disponible pour multiclasse
print("\n⚠️  Note: L'AUC n'est pas calculée pour la classification multiclasse")
print("\n📊 Résultats baseline (target ordered):")
pd.DataFrame(results_acc_ord).round(3)

🚀 Entraînement des modèles (target ordered)...

   RandomForest    → F1=0.491 | Accuracy=0.430
   XGBoost         → F1=0.644 | Accuracy=0.693

⚠️  Note: L'AUC n'est pas calculée pour la classification multiclasse

📊 Résultats baseline (target ordered):


,model,accuracy,precision,recall,f1
0,RandomForest,0.430,0.605,0.430,0.491
1,XGBoost,0.693,0.657,0.693,0.644


In [9]:
# Matrices de confusion pour ACCIDENT ordered (multi-classes)
labels_ordered = ["Indemne (0)", "Léger (1)", "Hospitalisé (2)", "Tué (3)"]

for name, preds in predictions_acc_ord.items():
    print(f"\n{'=' * 50}")
    print(f"MATRICE DE CONFUSION - ACCIDENT ordered - {name}")
    print("=" * 50)

    # Matrice de confusion
    plot_confusion_matrix(y_te_acc_ord, preds["y_pred"], f"ACCIDENT ordered - {name}", labels_ordered)


MATRICE DE CONFUSION - ACCIDENT ordered - RandomForest



MATRICE DE CONFUSION - ACCIDENT ordered - XGBoost


---

## 5. Comparaison Binary vs Ordered

### Objectif

Justifier le choix du target **binaire** sur le target **ordonné** en comparant :
- Les performances (F1, Accuracy, Precision, Recall)
- La distribution des classes
- L'exploitabilité des résultats pour l'API

### Hypothèse

Le target binaire devrait obtenir de meilleures performances car :
1. Problème plus simple (2 classes vs 4)
2. Classes moins déséquilibrées
3. Frontière de décision plus nette

In [10]:
# Comparaison des performances Binary vs Ordered - ACCIDENT uniquement
print("=" * 70)
print("COMPARAISON BINARY vs ORDERED - ACCIDENT - TOUTES LES METRIQUES")
print("=" * 70)


def get_all_metrics(y_true, y_pred, y_proba=None, is_binary=True):
    """Calcule toutes les metriques disponibles"""
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted", zero_division=0),
        "Recall": recall_score(y_true, y_pred, average="weighted", zero_division=0),
        "F1": f1_score(y_true, y_pred, average="weighted", zero_division=0),
    }

    if is_binary:
        cm = confusion_matrix(y_true, y_pred)
        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()
            metrics["Specificity"] = tn / (tn + fp) if (tn + fp) > 0 else 0
        metrics["MCC"] = matthews_corrcoef(y_true, y_pred)
        if y_proba is not None:
            y_score = y_proba[:, 1] if len(y_proba.shape) > 1 else y_proba
            metrics["AUC"] = roc_auc_score(y_true, y_score)
    else:
        metrics["Specificity"] = "-"
        metrics["MCC"] = "-"
        metrics["AUC"] = "-"

    return metrics


# Calculer les metriques pour ACCIDENT
comparison_data = []

# ACCIDENT Binary
m = get_all_metrics(
    y_test_acc,
    predictions_acc_bin["RandomForest"]["y_pred"],
    predictions_acc_bin["RandomForest"]["y_proba"],
    is_binary=True,
)
m["Target"] = "Binary (2 classes)"
comparison_data.append(m)

# ACCIDENT Ordered
m = get_all_metrics(
    y_te_acc_ord,
    predictions_acc_ord["RandomForest"]["y_pred"],
    predictions_acc_ord["RandomForest"]["y_proba"],
    is_binary=False,
)
m["Target"] = "Ordered (4 classes)"
comparison_data.append(m)

df_comp = pd.DataFrame(comparison_data)
cols_order = ["Target", "Accuracy", "Precision", "Recall", "Specificity", "F1", "AUC", "MCC"]
print("\nTABLEAU COMPARATIF ACCIDENT:")
print(df_comp[cols_order].to_string(index=False))

# Calcul de la difference
print("\nDIFFERENCE DE PERFORMANCE (Binary - Ordered):")
print("-" * 60)
for metric in ["Accuracy", "Precision", "Recall", "F1"]:
    bin_val = df_comp[df_comp["Target"].str.contains("Binary")][metric].values[0]
    ord_val = df_comp[df_comp["Target"].str.contains("Ordered")][metric].values[0]
    diff = bin_val - ord_val
    print(f"  {metric:12}: Binary={bin_val:.3f} | Ordered={ord_val:.3f} | Delta={diff:+.3f}")

COMPARAISON BINARY vs ORDERED - ACCIDENT - TOUTES LES METRIQUES

TABLEAU COMPARATIF ACCIDENT:
             Target  Accuracy  Precision   Recall Specificity       F1       AUC       MCC
 Binary (2 classes)  0.683095   0.681950 0.683095    0.760082 0.682501  0.705464  0.302128
Ordered (4 classes)  0.430210   0.604949 0.430210           - 0.491267         -         -

DIFFERENCE DE PERFORMANCE (Binary - Ordered):
------------------------------------------------------------
  Accuracy    : Binary=0.683 | Ordered=0.430 | Delta=+0.253
  Precision   : Binary=0.682 | Ordered=0.605 | Delta=+0.077
  Recall      : Binary=0.683 | Ordered=0.430 | Delta=+0.253
  F1          : Binary=0.683 | Ordered=0.491 | Delta=+0.191


In [11]:
# Graphique comparatif Binary vs Ordered - ACCIDENT
all_metrics = ["Accuracy", "Precision", "Recall", "F1"]

# Barres groupees
acc_bin = df_comp[df_comp["Target"].str.contains("Binary")].iloc[0]
acc_ord = df_comp[df_comp["Target"].str.contains("Ordered")].iloc[0]

fig = go.Figure()

fig.add_trace(
    go.Bar(
        name="Binary (2 classes)",
        x=all_metrics,
        y=[acc_bin[m] for m in all_metrics],
        marker_color="#2ecc71",
        text=[f"{acc_bin[m]:.3f}" for m in all_metrics],
        textposition="outside",
    )
)

fig.add_trace(
    go.Bar(
        name="Ordered (4 classes)",
        x=all_metrics,
        y=[acc_ord[m] for m in all_metrics],
        marker_color="#e74c3c",
        text=[f"{acc_ord[m]:.3f}" for m in all_metrics],
        textposition="outside",
    )
)

fig.update_layout(
    title="Comparaison Binary vs Ordered - ACCIDENT",
    barmode="group",
    height=450,
    width=700,
    yaxis_range=[0, 1.1],
    xaxis_title="Metrique",
    yaxis_title="Score",
)
fig.show()

In [12]:
# Analyse des distributions de classes - ACCIDENT
print("=" * 70)
print("ANALYSE DES DISTRIBUTIONS DE CLASSES - ACCIDENT")
print("=" * 70)

print("\nDistribution ACCIDENT grav_ordered:")
dist_acc = y_acc_ord.value_counts().sort_index()
for idx, count in dist_acc.items():
    pct = count / len(y_acc_ord) * 100
    label = ["Indemne", "Blesse leger", "Hospitalise", "Tue"][idx]
    bar = "#" * int(pct / 2)
    print(f"  {idx} ({label:15}): {count:6} ({pct:5.1f}%) {bar}")

print("\nDistribution ACCIDENT grav_binary:")
dist_bin = y_acc_bin.value_counts().sort_index()
for idx, count in dist_bin.items():
    pct = count / len(y_acc_bin) * 100
    label = ["Non grave", "Grave"][idx]
    bar = "#" * int(pct / 2)
    print(f"  {idx} ({label:15}): {count:6} ({pct:5.1f}%) {bar}")

print("\nConstat:")
print("  - En ordered: classes tres desequilibrees (Indemne < 1%, Tue ~ 6%)")
print("  - En binary: meilleur equilibre (~35% grave)")
print("  - Le modele ordered a du mal a predire les classes minoritaires")

ANALYSE DES DISTRIBUTIONS DE CLASSES - ACCIDENT

Distribution ACCIDENT grav_ordered:
  0 (Indemne        ):    627 (  0.2%) 
  1 (Blesse leger   ): 170222 ( 64.6%) ################################
  2 (Hospitalise    ):  78023 ( 29.6%) ##############
  3 (Tue            ):  14484 (  5.5%) ##

Distribution ACCIDENT grav_binary:
  0 (Non grave      ): 170849 ( 64.9%) ################################
  1 (Grave          ):  92507 ( 35.1%) #################

Constat:
  - En ordered: classes tres desequilibrees (Indemne < 1%, Tue ~ 6%)
  - En binary: meilleur equilibre (~35% grave)
  - Le modele ordered a du mal a predire les classes minoritaires


### 5.5 Conclusion

### Conclusion : Pourquoi privilégier le target binaire ?

| Critère | Binary (2 classes) | Ordered (4 classes) |
|---------|-------------------|---------------------|
| **F1-Score** | ~0.65-0.70 | ~0.45-0.55 |
| **Interprétabilité** | Simple : Grave / Non grave | Complexe : 4 niveaux |
| **Déséquilibre** | Modéré (30-40% grave) | Sévère (classe "Tué" < 5%) |
| **Utilité API** | Décision claire | Nuance peu exploitable |
| **AUC disponible** | Oui | Non (multi-classes) |

**Raisons du choix binaire :**

1. **Performances nettement supérieures** : F1 +15-20 points
2. **Classes très déséquilibrées** en ordered : le modèle prédit mal "Hospitalisé" et "Tué"
3. **Cas d'usage API** : la distinction Grave/Non-grave suffit pour déclencher les secours
4. **Matrices de confusion** : en ordered, confusion massive entre classes adjacentes

**Les modèles ordered restent disponibles** (`model_*_ordered.joblib`) pour des analyses plus fines si nécessaire.

---

---

## 6. Feature Importance

### Objectif

Identifier les **variables les plus prédictives** de la gravité d'un accident.

### Méthode

RandomForest calcule l'importance des features basée sur la **réduction moyenne d'impureté** (Gini) apportée par chaque variable lors des splits.

### Interprétation attendue

Les features les plus importantes devraient être :
- **Localisation** (agg) : La géographie influence fortement la gravité
- **Vitesse** (vma) : Plus la vitesse autorisée est élevée, plus le risque de gravité augmente
- **Type de véhicules** (impl_*) : Les piétons et 2RM sont plus vulnérables

In [13]:
# Feature importance pour ACCIDENT - utilise le meilleur modele binary
# Le meilleur modele est stocke dans models_acc ou predictions_acc_bin

# Recuperer le modele RF entraine
best_model_key = "RandomForest"  # ou 'XGBoost' selon les resultats
if best_model_key in models_acc:
    best_acc_model = models_acc[best_model_key]

    # Extraire le modele du pipeline
    if hasattr(best_acc_model, "named_steps"):
        model = best_acc_model.named_steps.get("model")

        if hasattr(model, "feature_importances_"):
            importance = pd.DataFrame({"feature": X_acc.columns, "importance": model.feature_importances_}).sort_values(
                "importance", ascending=False
            )

            fig = px.bar(
                importance.head(15),
                x="importance",
                y="feature",
                orientation="h",
                title="Top Features - ACCIDENT (grav_binary)",
            )
            fig.update_layout(yaxis={"categoryorder": "total ascending"}, height=500, width=700)
            fig.show()
        else:
            print("Le modele n'a pas d'attribut feature_importances_")
    else:
        print("Le modele n'est pas un Pipeline")
else:
    print(f"Modele {best_model_key} non trouve dans models_acc")

---

## 7. Sauvegarde du modèle baseline

### Pourquoi sauvegarder maintenant ?

Avant le fine-tuning, on sauvegarde le modèles baseline pour :
1. Avoir un point de comparaison
2. Pouvoir revenir en arrière si le fine-tuning dégrade les performances
3. Disposer d'un modèle fonctionnel rapidement

### Format de sauvegarde

- **joblib** : Format standard sklearn, rapide et compact
- **Pipeline complet** : Inclut preprocessing + modèle (prêt à l'emploi)

In [ ]:
os.makedirs("models", exist_ok=True)

# Modeles ACCIDENT de base (utiliser les modeles entraines en section 4)
# Sauvegarder le meilleur modele binary
if "models_acc" in dir() and "XGBoost" in models_acc:
    # XGBoost a de meilleures performances
    joblib.dump(models_acc["XGBoost"], "models/model_accident_binary.joblib")
    joblib.dump(X_acc.columns.tolist(), "models/features_accident.joblib")
    print("Sauvegarde: model_accident_binary.joblib (XGBoost)")
elif "models_acc" in dir() and "RandomForest" in models_acc:
    joblib.dump(models_acc["RandomForest"], "models/model_accident_binary.joblib")
    joblib.dump(X_acc.columns.tolist(), "models/features_accident.joblib")
    print("Sauvegarde: model_accident_binary.joblib (RandomForest)")

print("\n=== Modeles ACCIDENT de base sauvegardes ===")

Sauvegarde: model_accident_binary.joblib (XGBoost)

=== Modeles ACCIDENT de base sauvegardes ===


: 

---

## Suite

Le **fine-tuning** des modèles (HyperOpt, comparaison des algorithmes, sélection finale) se trouve dans le notebook **3-b-fine-tuning.ipynb**.